In [7]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as rnd
from scipy.stats import norm
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from skimage import io
from PIL import Image
from io import BytesIO
import requests
from skimage.util import view_as_windows
from stepmix.stepmix import StepMix
from stepmix.utils import get_mixed_descriptor

In [2]:
# Importation des données
star_data = pd.read_csv ('data_star.csv')

In [3]:
condition_couleur = [
    star_data["Star color"] == "Red",
    star_data["Star color"] == "Blue",
    star_data["Star color"] == "Blue White",
    star_data["Star color"] == "Yellow White",
    star_data["Star color"] == "White",
    star_data["Star color"] == "Yellowhish",
    star_data["Star color"] == "Orange",
    star_data["Star color"] == "Orange Red",
]

categories_couleur = [10, 4, 3, 1, 0, 2, 8, 9]

star_data["Couleur categ"] = np.select(condition_couleur, categories_couleur)
print(star_data[['Couleur categ']].value_counts())

Couleur categ
10               112
4                 56
3                 41
0                 15
1                 13
8                  2
9                  1
Name: count, dtype: int64


In [4]:
star_data.replace({'Spectral Class':{'M':0, 'A':1, 'B':2, 'F':1, 'O':2, 'K':0, 'G':1 }}, inplace = True)

/var/folders/vv/yd9tg_vd01b85m5mbpmc1v3r0000gn/T/ipykernel_44390/2265824004.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  star_data.replace({'Spectral Class':{'M':0, 'A':1, 'B':2, 'F':1, 'O':2, 'K':0, 'G':1 }}, inplace = True)


In [6]:
star_data_model = star_data.drop(columns = ['Star category'])
star_data_model = star_data_model.drop(columns = ['Star color'])

In [9]:
mixed_data, mixed_descriptor = get_mixed_descriptor(
    dataframe = star_data_model,
    continuous = ['Temperature (K)', 'Luminosity (L/Lo)', 'Radius (R/Ro)', 'Absolute magnitude (Mv)'],
    categorical = ['Star type', 'Couleur categ', 'Spectral Class']
)

In [10]:
model = StepMix(n_components = 6, measurement = mixed_descriptor, verbose = 1, random_state = 123)
# Fit model
model.fit(star_data_model)

# Class predictions
Y_predict = model.predict(star_data_model)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00, 98.31it/s, max_LL=-5.11e+3, max_avg_LL=-21.3]

MODEL REPORT
    Measurement model parameters
          model_name                             continuous                                                                     
          class_no                                        0            1             2             3             4             5
          param       variable                                                                                                  
          covariances Absolute magnitude (Mv)  7.614000e-01       8.0133  4.281400e+00  4.076600e+00  1.132800e+01  1.182000e+00
                      Luminosity (L/Lo)        3.166506e+10       0.0000  5.431693e+07  1.579766e+10  1.342000e-01  6.136677e+10
                      Radius (R/Ro)            8.594655e+02       0.0261  3.761700e+00  3.549362e+05  1.484000e-01  1.141699e+05
                      Temperature (K)          9.945883e+07  109698.1248  4.655226e+07  1.832028e+06  3.081413e+07  7.915368e+07
          means       Absolute magnitude (Mv) -6.04

In [13]:
classe_array = np.array(star_data['Star category'])
Y_pred_array = np.array(Y_predict)
df = pd.DataFrame({
    'prediction' : Y_pred_array,
    'classe' : classe_array
})
print(df)
df.to_csv("prediction_classes2.csv", index=False)

     prediction       classe
0             1  Brown Dwarf
1             1  Brown Dwarf
2             1  Brown Dwarf
3             1  Brown Dwarf
4             1  Brown Dwarf
..          ...          ...
235           5   Hypergiant
236           5   Hypergiant
237           3   Hypergiant
238           3   Hypergiant
239           5   Hypergiant

[240 rows x 2 columns]
